In [66]:
import numpy as np
import pandas as pd

In [67]:
movies=pd.read_csv('tmdb_5000_movies.csv')

In [68]:
from sklearn.preprocessing import LabelEncoder
movies.iloc[:,9]=LabelEncoder().fit_transform(movies.iloc[:,9])#converting to nubmer for genric type

In [69]:
new_movie=movies['production_companies'].value_counts()
new_movie=new_movie[new_movie>=1]

In [70]:
new_rate=movies['vote_average'].value_counts()
new_rate=new_rate[new_rate>20]
new_rate.head()

6.5    216
6.0    216
6.7    213
6.3    207
6.1    201
Name: vote_average, dtype: int64

In [71]:
movies=movies[movies['production_companies'].isin(new_movie.index)][['production_companies','title','vote_average','vote_count']]
movies.head(6)

,production_companies,title,vote_average,vote_count
0,1392,Avatar,7.2,11800
1,3536,Pirates of the Caribbean: At World's End,6.9,4500
2,526,Spectre,6.3,4466
3,1559,The Dark Knight Rises,7.6,9106
4,3573,John Carter,6.1,2124
5,564,Spider-Man 3,5.9,3576


In [72]:
movies=movies[movies['vote_average'].isin(new_rate.index)]
movies.head()

,production_companies,title,vote_average,vote_count
0,1392,Avatar,7.2,11800
1,3536,Pirates of the Caribbean: At World's End,6.9,4500
2,526,Spectre,6.3,4466
3,1559,The Dark Knight Rises,7.6,9106
4,3573,John Carter,6.1,2124


In [73]:
new_movies=movies.groupby('production_companies').agg('count')['vote_count'].reset_index().rename({'vote_count':'count'},axis=1)
movies=movies.merge(new_movies,on='production_companies')
movies.head()

,production_companies,title,vote_average,vote_count,count
0,1392,Avatar,7.2,11800,1
1,3536,Pirates of the Caribbean: At World's End,6.9,4500,2
2,3536,Pirates of the Caribbean: Dead Man's Chest,7.0,5246,2
3,526,Spectre,6.3,4466,1
4,1559,The Dark Knight Rises,7.6,9106,1


In [74]:
movies=movies[movies['vote_average']>5.5]
movies=movies[movies['vote_count']>100]
movies.head()

,production_companies,title,vote_average,vote_count,count
0,1392,Avatar,7.2,11800,1
1,3536,Pirates of the Caribbean: At World's End,6.9,4500,2
2,3536,Pirates of the Caribbean: Dead Man's Chest,7.0,5246,2
3,526,Spectre,6.3,4466,1
4,1559,The Dark Knight Rises,7.6,9106,1


In [75]:
final_pivot=movies.pivot_table(values='vote_average',index='title',columns='production_companies').fillna(0)
final_pivot.head()

production_companies,0,1,2,4,5,6,9,11,12,15,...,3665,3666,3667,3668,3670,3671,3675,3678,3687,3688
title,,,,,,,,,,,,,,,,,,,,,
(500) Days of Summer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Cloverfield Lane,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Things I Hate About You,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11:14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12 Rounds,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [76]:
from scipy.sparse import csr_matrix
movies_sparse=csr_matrix(final_pivot)

In [77]:
from sklearn.neighbors import NearestNeighbors
nn=NearestNeighbors(metric='cosine',algorithm='brute')
nn.fit(movies_sparse)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [78]:
np.where(final_pivot.index=='The Fast and the Furious')[0][0]

1971

In [79]:
distance,suggestion=nn.kneighbors(final_pivot.iloc[1971,:].values.reshape(1,-1),n_neighbors=6)

In [80]:
suggestion=final_pivot.index[suggestion].ravel()

In [81]:
for i in range(len(suggestion)):
    print([suggestion[i]])

['The Fast and the Furious']
['Star Trek VI: The Undiscovered Country']
['Star Trek Into Darkness']
['Star Trek: First Contact']
['Star Trek: Insurrection']
['Star Trek V: The Final Frontier']
